In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import torch 
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
torch.cuda.set_per_process_memory_fraction(0.7, device = 0)
os.chdir('../src/raw_data')
warnings.filterwarnings('ignore')
torch.manual_seed(52)
np.random.seed(52)

In [3]:
label_df = pd.read_csv('ground_truth.csv', sep = ';')
label_df

,file,mark,recovery,drop
0,00e03657-8e1e-4c8c-a724-1d3c77b48510,"[0.0,235.9225,237.06666666666666,2076.06055555...","[[2419.9805555555554,2437.4241666666667],[3177...","[[3453.6875,3763.9605555555554]]"
1,00e4dba2-36d2-42b4-beb1-c55aed75f506,"[0.0,7979.234444444444,13284.465,19439.8005555...",[],"[[13284.465,19439.800555555557]]"
2,00f035b7-ad7a-4f30-9081-522a3c10805b,"[0.0,42.75,2438.3330555555553]",[],"[[0.0,42.75]]"
3,01a0c034-6afc-4e73-95fa-621f702a0b7d,"[0.0,491.98305555555555,1439.9830555555557,154...",[],"[[0.0,491.98305555555555]]"
4,01a530d3-6496-4515-9fbb-4f44e298fd29,"[0.0,1287.0341666666666,1288.0483333333334,156...",[],"[[4920.376666666667,6208.231666666667]]"
...,...,...,...,...
95,1dfaf03c-e297-4d92-a0bf-40b1a829391f,"[0.0,7.4,7.933055555555556,14.466666666666667,...",[],[]
96,1e149fbd-41c6-4779-b87d-c5dc17fbb4c0,"[0.0,635.3127777777778]",[],"[[0.0,635.3127777777778]]"
97,1e19b77c-8a0e-4749-a384-9c1e679035bf,"[0.0,82.16555555555556,216.66027777777776,229....",[],[]
98,1e4b4c18-1e32-45eb-917a-5760e33fbaca,"[0.0,1217.8258333333333,1223.6030555555556,125...","[[9541.77638888889,10288.5075]]","[[10339.343055555555,10739.613055555556],[1311..."


In [4]:
SEQUENCE_LENGTH = 100  
BATCH_SIZE = 32
NOISE_FACTOR = 0.05
TRAIN_DIR = "../train_reduced/"
CHECKPOINT_PATH = '../../models/checkpoint.pth'

In [5]:
def load_raw_data(file_path: str, sequence_length = SEQUENCE_LENGTH, noise_factor = NOISE_FACTOR):
    df = pd.read_csv(file_path, sep="\\s+", names=["time", "pressure"])  
    
    if df.empty or "pressure" not in df:
        return torch.empty(0), torch.empty(0) 
    
    scaler = MinMaxScaler()
    df["pressure"] = scaler.fit_transform(df[["pressure"]]) 

    sequences, noisy_sequences = [], []
    
    for i in range(len(df) - sequence_length):
        seq = df["pressure"].iloc[i : i + sequence_length].values
        noisy_seq = seq + noise_factor * np.random.normal(0, 1, seq.shape) 
        
        sequences.append(seq)
        noisy_sequences.append(noisy_seq)

    return torch.tensor(noisy_sequences, dtype=torch.float32).unsqueeze(-1), torch.tensor(sequences, dtype=torch.float32).unsqueeze(-1)

In [6]:
class TorchTypeDataset(Dataset):
    def __init__(self, file_paths, sequence_length = SEQUENCE_LENGTH, noise_factor = NOISE_FACTOR):
        self.noisy_data, self.clean_data = [], []
        
        for file_path in file_paths:
            if os.path.exists(file_path): 
                noisy, clean = load_raw_data(file_path, sequence_length, noise_factor)

                if noisy.shape[0] > 1 and clean.shape[0] > 1:  
                    noisy = noisy.unsqueeze(-1) if noisy.dim() == 2 else noisy
                    clean = clean.unsqueeze(-1) if clean.dim() == 2 else clean

                    self.noisy_data.append(noisy)
                    self.clean_data.append(clean)

        self.noisy_data = torch.cat(self.noisy_data, dim=0)  
        self.clean_data = torch.cat(self.clean_data, dim=0)

    def __len__(self):
        return len(self.clean_data)

    def __getitem__(self, idx):
        return self.noisy_data[idx], self.clean_data[idx]

In [7]:
class CharbonnierLoss(nn.Module):
    def __init__(self, epsilon = 1e-3):
        super(CharbonnierLoss, self).__init__()
        self.epsilon = epsilon
    
    def forward(self, x, y):
        return torch.mean(torch.sqrt((x - y) ** 2 + self.epsilon ** 2))

class Attention(nn.Module):
    def __init__(self, dim):
        super(Attention, self).__init__()
        self.attn = nn.MultiheadAttention(embed_dim = dim, num_heads = 4, batch_first = True)
    
    def forward(self, x):
        attn_output, _ = self.attn(x, x, x)
        return attn_output

class DAE(nn.Module):
    def __init__(self, input_dim = 1, hidden_dim = 64, bottleneck_dim = 32):
        super(DAE, self).__init__()  
      
        self.conv1 = nn.Conv1d(in_channels = input_dim, out_channels = 32, kernel_size = 5, padding = 2)
        self.lstm1 = nn.LSTM(input_size = 32, hidden_size = hidden_dim, batch_first = True, bidirectional = False)
        self.attn = Attention(hidden_dim)
        self.bottleneck = nn.Linear(hidden_dim, bottleneck_dim)
        
        self.fc = nn.Linear(bottleneck_dim, hidden_dim)
        self.lstm2 = nn.LSTM(input_size = hidden_dim, hidden_size = hidden_dim, batch_first = True, bidirectional = False)
        self.conv2 = nn.Conv1d(in_channels = hidden_dim, out_channels = input_dim, kernel_size = 5, padding = 2)
    
    def forward(self, x):
        x = self.conv1(x.transpose(1, 2))
        x = F.relu(x).transpose(1, 2)
        x, _ = self.lstm1(x)
        x = self.attn(x)
        x = self.bottleneck(x)
        
        x = self.fc(x)
        x, _ = self.lstm2(x)
        x = self.conv2(x.transpose(1, 2)).transpose(1, 2)
        return x

In [8]:
def DAE_train(model, dataloader, epochs = 50, lr = 1e-3, device='cuda', save_path = "../../models/dae_checkpoint.pth", final_save_path = "../../models/dae_final.pth"):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr = lr)
    criterion = CharbonnierLoss()

    best_loss = float("inf")  

    for epoch in range(epochs):
        total_loss = 0.0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch + 1} / {epochs}") 
        
        for noisy, clean in progress_bar:
            noisy, clean = noisy.to(device), clean.to(device)
            optimizer.zero_grad()
            output = model(noisy)
            loss = criterion(output, clean)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            progress_bar.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch + 1} / {epochs}, Avg Loss: {avg_loss:.6f}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), save_path)  
            print(f"Model checkpoint saved at {save_path} (Loss improved: {best_loss:.6f})")

    torch.save(model.state_dict(), final_save_path)
    print(f"Final model saved at {final_save_path}")

    return model

In [9]:
def run_inference(model, input_tensor, scaler, chunk_size=500, device='cuda'):
    model.to(device)
    input_tensor = input_tensor.to(device)
    denoised_chunks = []
    
    with torch.no_grad():
        for chunk in torch.split(input_tensor, chunk_size, dim=0):
            denoised_chunk = model(chunk)
            denoised_chunks.append(denoised_chunk.cpu())
    
    denoised_output = torch.cat(denoised_chunks, dim=0)
    return scaler.inverse_transform(denoised_output.numpy().squeeze())


In [10]:
file_paths = [os.path.join(TRAIN_DIR, file) for file in label_df['file'][:100] if file in os.listdir(TRAIN_DIR)]

dataset = TorchTypeDataset(file_paths)
dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = True)

In [11]:
for noisy_batch, clean_batch in dataloader:
    print("Noisy batch shape:", noisy_batch.shape, end = ' ')  
    print("Clean batch shape:", clean_batch.shape)
    break

Noisy batch shape: torch.Size([32, 100, 1]) Clean batch shape: torch.Size([32, 100, 1])


In [12]:
dae_model = DAE()

if os.path.exists(CHECKPOINT_PATH):
    dae_model.load_state_dict(torch.load(CHECKPOINT_PATH))
    print(f"Checkpoint loaded from {CHECKPOINT_PATH}")

trained_model = DAE_train(dae_model, dataloader, epochs = 2, lr=1e-3, device='cuda')

Epoch 1 / 2: 100%|██████████| 13539/13539 [01:47<00:00, 125.64it/s, loss=0.0133] 


Epoch 1 / 2, Avg Loss: 0.012767
Model checkpoint saved at ../../models/dae_checkpoint.pth (Loss improved: 0.012767)


Epoch 2 / 2: 100%|██████████| 13539/13539 [01:48<00:00, 124.89it/s, loss=0.00882]

Epoch 2 / 2, Avg Loss: 0.010475
Model checkpoint saved at ../../models/dae_checkpoint.pth (Loss improved: 0.010475)
Final model saved at ../../models/dae_final.pth


In [29]:
trained_model.eval()
trained_model.to(device)

def preprocess_file(file_path, sequence_length=100):
    df = pd.read_csv(file_path, sep="\\s+", names=["time", "pressure"])
    
    scaler = MinMaxScaler()
    df["pressure"] = scaler.fit_transform(df[["pressure"]])
    
    sequences = []
    for i in range(len(df) - sequence_length):
        seq = df["pressure"].iloc[i : i + sequence_length].values
        sequences.append(seq)

    input_tensor = torch.tensor(np.array(sequences), dtype=torch.float32).unsqueeze(-1)  # (batch, seq_len, 1)

    print(f"Размер `df`: {df.shape}")
    print(f"Размер `sequences`: {len(sequences)}")
    print(f"Размер `input_tensor`: {input_tensor.shape}")

    if "time" in df and len(df["time"]) >= len(sequences):
        time_values = df["time"][:len(sequences)].values
        return input_tensor, scaler, time_values
    else:
        return input_tensor, scaler



In [31]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

test_file = "../raw_data/test/1cbce6e5-9f0b-419f-9527-7add4e255217" 
result = preprocess_file(test_file)

Размер `df`: (54215, 2)
Размер `sequences`: 54115
Размер `input_tensor`: torch.Size([54115, 100, 1])


In [ ]:



if len(result) == 3:
    input_tensor, scaler, time_values = result
else:
    input_tensor, scaler = result
    time_values = None

input_tensor = input_tensor.to(device)

chunk_size = 500  
denoised_chunks = []

with torch.no_grad():
    
    for chunk in torch.split(input_tensor, chunk_size, dim=0):
        denoised_chunk = dae_model(chunk.to(device))  
        denoised_chunks.append(denoised_chunk.cpu()) 

denoised_output = torch.cat(denoised_chunks, dim=0)
denoised_data = scaler.inverse_transform(denoised_output.numpy().squeeze())

Размер входа в модель: torch.Size([54115, 100, 1])
Размер выхода из модели: torch.Size([54115, 100, 1])


In [16]:
denoised_data

array([[45.770573, 45.69212 , 45.374577, ..., 46.061058, 46.190327,
        46.160522],
       [45.786808, 45.708652, 45.3913  , ..., 46.070038, 46.200497,
        46.169773],
       [45.80486 , 45.72792 , 45.409447, ..., 46.079105, 46.210453,
        46.179783],
       ...,
       [27.345001, 27.270355, 27.285076, ..., 27.168692, 27.334549,
        27.47717 ],
       [27.344183, 27.269205, 27.28544 , ..., 27.168417, 27.334524,
        27.477148],
       [27.34387 , 27.2686  , 27.285686, ..., 27.166836, 27.332396,
        27.47537 ]], dtype=float32)

In [17]:
file_path = "../raw_data/test/1cbce6e5-9f0b-419f-9527-7add4e255217" 
orig_df = pd.read_csv(file_path, sep="\\s+", names=["time", "pressure"])
pred_df = orig_df
pred_df['pressure'] = denoised_data

ValueError: Length of values (54115) does not match length of index (54215)

In [27]:
# === Готовим тензор первых 100 значений ===
first_100_tensor = torch.tensor(orig_df["pressure"].values[:100], dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)

print(f"Размер orig_df: {orig_df.shape}")
print(f"Размер denoised_data: {denoised_data.shape}")

print(f"Размер first_100_tensor (до модели): {first_100_tensor.shape}")

# === Прогоняем через модель ===
with torch.no_grad():
    first_100_denoised = dae_model(first_100_tensor).cpu().numpy().squeeze()

print(f"Размер first_100_denoised (после модели): {first_100_denoised.shape}")

# === Приводим `first_100_denoised` к (100, 1) ===
first_100_denoised = first_100_denoised.reshape(-1, 1)

# === Приводим `denoised_data` к (N, 1), если это нужно ===
denoised_data = denoised_data.reshape(-1, 1)


print(f"Размер first_100_denoised (после reshape): {first_100_denoised.shape}")
print(f"Размер denoised_data (после reshape): {denoised_data.shape}")

# === Объединяем тензоры ===
full_denoised = np.concatenate([first_100_denoised, denoised_data], axis=0)

# === Добавляем в DataFrame ===
orig_df["denoised_pressure"] = full_denoised


Размер orig_df: (54215, 2)
Размер denoised_data: (5411500, 1)
Размер first_100_tensor (до модели): torch.Size([1, 100, 1])
Размер first_100_denoised (после модели): (100,)
Размер first_100_denoised (после reshape): (100, 1)
Размер denoised_data (после reshape): (5411500, 1)


ValueError: Length of values (5411600) does not match length of index (54215)